In [63]:
import pandas as pd
import torch
import torch.nn as nn

from io import StringIO

In [64]:
notearray_path = "../jrp.txt"

In [8]:
with open(notearray_path, "r") as in_text:
    lines = in_text.readlines()

In [9]:
for i in range(10):
    print("Line",i,lines[i])

Line 0 % !!!COM:	Anonymous

Line 1 % !!!OTL:	De sartor nui sian maestri

Line 2 % !!!AGN:	Song; Carnival song

Line 3 % !!!SCT:	Ano3237

Line 4 % !!!SCA:	Ano3237

Line 5 % !!!SMS:	Canto dei sartori (Perugia G.20)

Line 6 % !!!folio:	

Line 7 % !!!voices:	4

Line 8 %%idx	%%line	%%bar	%%mdur	%%bdur	%%beat	%%abs	%%b40	%%attk	%%last	%%next	%%b40	%%attk	%%last	%%next	%%b40	%%attk	%%last	%%next	%%b40	%%attk	%%last	%%next

Line 9 0	2000	3000	3100	3200	4000	5000	1040	7100	7200	7300	1040	7100	7200	7300	1040	7100	7200	7300	1040	7100	7200	7300



In [123]:
class NoteArray(torch.utils.data.Dataset):
    '''
    All the info about the note array
    '''
    
    def __init__(self,notearray_path):
        """
        initialize: get all the metadata and stuff
        """
        
        with open(notearray_path, "r") as notearray:
            """
            self.lines = whole notearray, line-by-line
            self.notelines = the music part of the notearray
            self.notelines_totals = the totals of each column
            self.df, self.df_totals = df-ization of the above lines
            self.composer = composer
            self.title = title
            """
            
            self.lines = notearray.readlines()
            self.meta_lines = {
                ii:line for ii,line in enumerate(self.lines) if line.count('!') != 0
            }
            start_line_num = min(set(range(len(self.lines)))-set(list(self.meta_lines.keys())))
            end_line_num = max(set(range(len(self.lines))) - set(list(self.meta_lines.keys())))
            notelines =  self.lines[start_line_num : end_line_num+1]
            start_line = notelines[0]
            notelines[0] = start_line.replace('%','')
            
            # totals
            notelines_totals = notelines[:2]            
            
            # just the music itself
            del notelines[1]
            self.notelines = notelines
            
            cleaned_notearray = StringIO('\n'.join(self.notelines))
            cleaned_totals = StringIO('\n'.join(notelines_totals))
            
            self.df = pd.read_csv(cleaned_notearray, delimiter='\t')
            self.df_totals = pd.read_csv(cleaned_totals, delimiter='\t')
            
            # metadata
            meta_dic = {}
            for line in list(self.meta_lines.values()):
                try:
                    a,b = re.split(r'\t+', line.rstrip('\n'))
                    meta_dic[a] = b
                except:
                    continue
                
            try:
                self.num_voices = meta_dic['% !!!voices:']
            except:
                self.num_voices = meta_dic['% !!!voices-ORP:']
                
            self.composer = meta_dic['% !!!COM:']
            self.title = meta_dic['% !!!OTL:']
            try:
                self.genre = meta_dic['% !!!AGN:']
            except: 
                self.genre = None
            

In [124]:
note_array = NoteArray(notearray_path)

In [129]:
note_array.df_totals

,idx,line,bar,mdur,bdur,beat,abs,b40,attk,last,...,last.1,next.1,b40.2,attk.2,last.2,next.2,b40.3,attk.3,last.3,next.3
0,0,2000,3000,3100,3200,4000,5000,1040,7100,7200,...,7200,7300,1040,7100,7200,7300,1040,7100,7200,7300
